This notebook is intented to create and explore different ratio calulations. Resulting processes/functions will be copied to ./dat/

In [1]:
# Set matplotlib for plotting in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

from fooof import FOOOF
from fooof import FOOOFGroup
import numpy as np
from utils.ratios import *

from fooof.utils import trim_spectrum
# Import some utilities for synthesizing some test data
from fooof.synth import gen_group_power_spectra, param_sampler, gen_power_spectrum

## Simulate Power Spectra

In [2]:
#Creates a single adhd simulated spectrum
# Alpha peaks on lower park of band to potentially bleed over to theta band

def gen_treatment_sample():
    sample = []
    
    sample.append(np.random.uniform(4, 7)) #Theta freq
    sample.append(np.random.uniform(.35,.75)) #Theta Amp
    sample.append(np.random.uniform(.25,1.05))#Theta SD
    
    sample.append(np.random.uniform(8, 10)) #slow Alpha freq
    sample.append(np.random.uniform(.25,.55)) #Alpha Amp
    sample.append(np.random.uniform(.25,1.05))#Alpha SD
    
    sample.append(np.random.uniform(18, 25)) #Beta freq
    sample.append(np.random.uniform(.4,.75)) #Beta Amp
    sample.append(np.random.uniform(.25,1.05))#Beta SD
    
    
    return sample

In [3]:
#Creats a single control simulated spectrum (Theta, slow peak alpha, beta)
def gen_control_sample():
    sample = []
    
    sample.append(np.random.uniform(4, 7)) #Theta freq
    sample.append(np.random.uniform(.35,.75)) #Theta Amp
    sample.append(np.random.uniform(.25,1.05))#Theta SD
    
    sample.append(np.random.uniform(8, 12)) #slow alpha freq
    sample.append(np.random.uniform(.25,.55)) #Alpha Amp
    sample.append(np.random.uniform(.25,1.05))#Alpha SD
    
    sample.append(np.random.uniform(18, 25)) #Beta freq
    sample.append(np.random.uniform(.4,.75)) #Beta Amp
    sample.append(np.random.uniform(.25,1.05))#Beta SD
    
    
    return sample

In [4]:
#Gen group of ADHD PSDs (slow alpha peak freq) + control PSD
bg = [0,1]

#100 trails for each adhd and control sim
treatmentFreq, treatmentPower, _ = gen_group_power_spectra(100, [1,50], bg,gen_treatment_sample(), nlvs=np.random.uniform(.005,.02)) 
contFreq, contPower, _ = gen_group_power_spectra(100, [1,50], bg, gen_control_sample(), nlvs=np.random.uniform(.005,.02))

treatment = FOOOFGroup(peak_width_limits=[1,8], min_peak_amplitude=0.05, max_n_peaks=3)
control = FOOOFGroup(peak_width_limits=[1,8], min_peak_amplitude=0.05, max_n_peaks=3)

In [5]:
treatment.fit(treatmentFreq,treatmentPower)
control.fit(contFreq,contPower)

In [6]:
#Gather peak info for both PSDs
treatment_peaks = treatment.get_all_data('peak_params')
control_peaks = control.get_all_data('peak_params')

## Calculate theta/beta ratios 
#### difference in power of central frequency of high and low frequency peak only

In [7]:
# treatment (Slow alpha peak)
treatmentRatioList = []
for i in range(len(treatment_peaks)-2):
    treatmentRatioList.append(treatment_peaks[i][1]/treatment_peaks[i+2][1])
    i+=2

In [8]:
#Sanity Check
treatmentRatioList[0]

1.5634799649328617

In [9]:
# control
controlRatioList = []
for i in range(len(control_peaks)-2):
    controlRatioList.append(control_peaks[i][1]/control_peaks[i+2][1])
    i+=2

In [10]:
#Sanity Check
controlRatioList[0]

0.645386665969139

In [11]:
#Difference in mean ratios of treatment sim and control sim
abs(np.mean(treatmentRatioList)-np.mean(controlRatioList))

0.012782942241229556

### Difference in - average low band power / average high band power (bands set)

In [12]:
treatment_ratio = calc_theta_beta_ratio(treatmentFreq,treatmentPower)
control_ratio = calc_theta_beta_ratio(contFreq,contPower)
print("treatment ratio: ",treatment_ratio)
print("control ratio: ", control_ratio)

treatment ratio:  7.41986245923321
control ratio:  4.529306734093322


In [13]:
treatment_ratio - control_ratio

2.8905557251398877

### Difference in - sum(powers in band)/bandwidth ratio

In [14]:
def calc_band_ratio_sum(freqs, psd, low_band_range, high_band_range):
    
    # Extract frequencies within each specified band
    _, low_band_powers = trim_spectrum(freqs, psd, low_band_range)
    _, high_band_powers = trim_spectrum(freqs, psd, high_band_range)
   
    # Divides each sum of each power spectrum by bandwidth.
    low_sum = sum(low_band_powers) / len(low_band_range) 
    high_sum = sum(high_band_powers) / len(low_band_range)
    
    # Average divided sums and find ratio
    return np.mean(low_sum)/np.mean(high_sum)
    

In [15]:
calc_band_ratio_sum(treatmentFreq,treatmentPower,[4,8],[15,30])


6.078205309211364

In [19]:
treatment?

In [33]:
treatment.power_spectra

array([[ 1.18668239e-02, -1.92804379e-01, -2.94855073e-01, ...,
        -1.67540823e+00, -1.68813444e+00, -1.70021281e+00],
       [ 2.78225947e-03, -1.73782530e-01, -3.11306558e-01, ...,
        -1.68598146e+00, -1.69649197e+00, -1.70474704e+00],
       [-1.77019066e-03, -1.74333976e-01, -2.85147135e-01, ...,
        -1.68678979e+00, -1.68997816e+00, -1.66527332e+00],
       ...,
       [ 4.10910998e-04, -1.80590796e-01, -2.87863762e-01, ...,
        -1.68988648e+00, -1.69780078e+00, -1.69523747e+00],
       [-3.42326951e-03, -1.76366884e-01, -3.05790656e-01, ...,
        -1.69816112e+00, -1.70775548e+00, -1.71037705e+00],
       [ 9.45671672e-03, -1.80745467e-01, -3.11506466e-01, ...,
        -1.67652821e+00, -1.69917847e+00, -1.67789527e+00]])

In [34]:
def calc_group_band_ratio(fg,low_band_range, high_band_range):
    """Calculate average power in band ratio 

    ----------
    fg : fooof group object used to find ratio
    low_band_range : list of [float, float]
        Band definition for the lower band.
    high_band_range : list of [float, float]
        Band definition for the upper band.

    Outputs
    -------
    ratios : list of floats
        Oscillation power ratios.
    """
    size = len(fg.get_results())
    res = []
    for i in range(size):
        res.append(calc_band_ratio(fg.freqs,fg.power_spectra[i], low_band_range, high_band_range))
    return res

def calc_group_cf_power_ratio(fg, low_band_range, high_band_range):
    """Calculate band ratio by finding the power of high and low central frequency

    ----------
    fg : fooof group object used to find ratio
    low_band_range : list of [float, float]
        Band definition for the lower band.
    high_band_range : list of [float, float]
        Band definition for the upper band.

    Outputs
    -------
    ratios : floats
        Oscillation power ratio.
    """
    size = len(fg.get_results())
    res = []
    for i in range(size):
        res.append(calc_cf_power_ratio(fg.get_fooof(i), low_band_range, high_band_range))
    return res
def calc_group_density_ratio(fg, low_band_range, high_band_range):
    """Calculate band ratio by summing power power in bands and dividing by bandwidth

    ----------
    fg : fooof object used to find ratio
    low_band_range : list of [float, float]
        Band definition for the lower band.
    high_band_range : list of [float, float]
        Band definition for the upper band.

    Outputs
    -------
    ratios : floats
        Oscillation power ratio.
    """
    size = len(fg.get_results())
    res = []
    for i in range(size):
        res.append(calc_density_ratio(fg.freqs,fg.power_spectra[i], low_band_range, high_band_range))
    return res

In [37]:
calc_group_density_ratio(treatment,[4,8],[15,30])

[0.09710265867839495,
 0.0971835805430876,
 0.09818214102122791,
 0.09758830914464968,
 0.09685531095421576,
 0.09703499273110072,
 0.09648615972620006,
 0.09736090312125935,
 0.09679579558653365,
 0.09737150182924464,
 0.09912909358242811,
 0.09665327215876127,
 0.09551610863416056,
 0.0969058048200659,
 0.09652846397895164,
 0.09775790770992052,
 0.0971428158044019,
 0.09745366934760862,
 0.09724864477705401,
 0.0971838649943476,
 0.09778689783083863,
 0.0969596810567379,
 0.09808100508651228,
 0.0972999413771641,
 0.09715496567396201,
 0.0968008096408031,
 0.09683201953989978,
 0.09751517758884554,
 0.09858235304602338,
 0.09709138591794439,
 0.09649006753045172,
 0.09794180058175786,
 0.09714568685540298,
 0.09653451891600907,
 0.09701032910720629,
 0.09647073351180621,
 0.09697071257429042,
 0.0967985561038022,
 0.09742532474538508,
 0.09703484788621737,
 0.09702166443337489,
 0.09738762770356885,
 0.09604638209282426,
 0.0986771762916046,
 0.09740795334067451,
 0.0971913776116397